In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM

In [2]:
def flux_blr(Hb1_blr_br_data,Hb2_blr_br_data,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = np.sum(Hb1_blr_br_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux2 = np.sum(Hb2_blr_br_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux = (flux1 + flux2)*(10**(-16))
    else:
        flux = flux1*(10**(-16)) 
    return flux

def Hb_blr_lum(z,flux):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    return Hb_lum
    
def ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br):
    ratio = (vel_sigma_OIII_br/vel_offset)
    return ratio

def line_shape_par(z,wav_min,wav_max,cor_popt):
    c = 3*(10**5)
    wave = np.arange(wav_min,wav_max,0.02) 
    cumsum = np.cumsum(OIII_wo_cont(wave,*cor_popt))
    norm_sum=cumsum/cumsum[-1]
    
    select = (norm_sum>0.1) & (norm_sum<0.9)
    w80_spec = wave[select][-1]-wave[select][0]
    plt.plot(wave[select],norm_sum[select],'-k')
    plt.xlim([wav_min+85,wav_max-85])
    plt.show()
    
    select =  (norm_sum>0.05) & (norm_sum<0.5)
    v5_spec = wave[select][0]-wave[select][-1]
    plt.plot(wave[select],norm_sum[select],'-b')
    plt.xlim([wav_min,wav_max])
    plt.show()
    
    [w80,v5] = [((w80_spec)/5006.8)*(c/(1+z)),((v5_spec)/5006.8)*(c/(1+z))]
    return w80,v5

def par_tab(obj,w80,v5,vel_offset,vel_sigma_OIII_br,ratio,flux,Hb_lum):
    par = [w80,v5,vel_offset,vel_sigma_OIII_br,ratio,flux,Hb_lum]
    column_names={'w80':0,'v5':1,'vel_offset':2,'vel_sigma_OIII_br':3,'ratio':4,'blr_flux':5,'log_Hb_lum':6}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[par[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s_par_table.fits'%(obj),overwrite=True)      
    


In [3]:
def algorithm_script(obj,z,broad2):
    (amp_Hb1,amp_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset) = par(obj)
    (Hb1_blr_br_data,Hb2_blr_br_data) = loadblr('subcube_par_%s.fits'%(obj))
    
    sigma_eff_Hb1 = line_width_recons(vel_sigma_Hb1,rest_line=4861.33,inst_res_fwhm=0)
    sigma_eff_Hb2 = line_width_recons(vel_sigma_Hb2,rest_line=4861.33,inst_res_fwhm=0) 
    flux = flux_blr(Hb1_blr_br_data,Hb2_blr_br_data,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
    Hb_lum = Hb_blr_lum(z,flux)
    
    cor_popt = np.array([amp_OIII5007,amp_OIII5007_br,vel_OIII,vel_sigma_OIII,vel_OIII_br,vel_sigma_OIII_br])
    
    (wav_min,wav_max) = wavlim(vel_OIII,vel_OIII_br)
    (w80,v5) = line_shape_par(z,wav_min,wav_max,cor_popt)
    
    ratio = ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br)
    par_tab(obj,w80,v5,vel_offset,vel_sigma_OIII_br,ratio,flux,Hb_lum)

In [4]:
z = {"HE0021-1819":0.053197,"HE0040-1105":0.041692 #,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143,"HE0253-1641":0.031588
    ,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009,"HE0433-1028":0.035550
    ,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986,"HE1029-1831":0.040261
    ,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1330-1013":0.022145,"HE1353-1917":0.035021
    ,"HE1417-0909":0.044,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

#z_remaining = {"HE2128-0221":0.05248,"HE1248-1356":0.01465}

broad2= {'HE0021-1819':False,'HE0040-1105':False #,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False,'HE0253-1641':True
        ,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True,'HE0433-1028':True
        ,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False,'HE1029-1831':True
        ,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1330-1013':True,'HE1353-1917':True
        ,'HE1417-0909':False,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':True}

objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

FileNotFoundError: [Errno 2] No such file or directory: 'HE0021-1819_central_fit.fits'

In [ ]:
hdu = fits.open('par_table.fits')
central_tab = hdu[1].data
central_columns = hdu[1].header
log_Hb_lum = central_tab.field('log_Hb_lum')
w80 = central_tab.field('W80')
plt.plot(log_Hb_lum,np.log10(w80),'kx')
plt.xlabel('log_Hb_blr(erg/s)')
plt.ylabel('log_w80(km/s)')

plt.show()